# Packages

### install packages

In [ ]:
# !pip install numpy as np
# !pip install sklearn
# !pip install matplotlib
# !pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### import packages

In [ ]:
import os
import json

import demoji

import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

# Uploading Data

### uploading stemmed tweets, vocabs and tokenization matrix

In [ ]:
data_folder = 'sample_data'
data_folder = os.path.join(os.curdir,data_folder)
stemmed_tweets_path = os.path.join(data_folder,'stemmed_tweets.json')
vocab_path = os.path.join(data_folder,'vocabs.json')
tokenization_path = os.path.join(data_folder,'tokenization_matrix.json')
stemmed_tweets = {}
vocabs = {}
tokenization_list = {}
with open(stemmed_tweets_path,encoding='utf-8') as f:
  stemmed_tweets = json.load(f)

with open(vocab_path,encoding='utf-8') as f:
  vocabs = json.load(f)

with open(tokenization_path, 'r') as f:
  tokenization_list = json.load(f)

train_input = stemmed_tweets["train_input"]
train_output = stemmed_tweets["train_output"]
test_input = stemmed_tweets["test_input"]
test_output = stemmed_tweets["test_output"]

vocab_to_index = vocabs["vocab_to_index"]
index_to_vocab = vocabs["index_to_vocab"]

tokenization_matrix = tokenization_list['tokenization_matrix']
test_data_tokenization_matrix = tokenization_list['test_data_tokenization_matrix']
train_data_tokenization_matrix = tokenization_list['train_data_tokenization_matrix']

print("train data",len(train_input),len(train_output))
print("test data",len(test_input),len(test_output))
print("vocabs",len(vocab_to_index),len(index_to_vocab))
print("all data",len(test_input) + len(train_input), "tokenization", len(tokenization_matrix))

train data 44861 44861
test data 11426 11426
vocabs 9285 9285
all data 56287 tokenization 56287


# Vectorization

### replace emojis with texts

In [ ]:
# train data first because dataset for tokenization were based in concatenation of (all_stemmed = pd.concat([selected_train_data,selected_test_data])) 
all_tweets = train_input + test_input
all_tweets_text_emojis = []
vocab_to_index_text_emojis = {key:value for (key,value) in vocab_to_index.items()}
emojis_list = {}
for tweet in all_tweets:
  emojis = demoji.findall(tweet)
  for emoji in emojis.keys():
    value = emojis.get(emoji).replace(" ","").replace("-","")
    if value == 'None' or value == None:
      print(emoji)
    tweet = tweet.replace(emoji,value)
    if value not in vocab_to_index_text_emojis:
      emoji_index = vocab_to_index_text_emojis.get(emoji)
      try:
        vocab_to_index_text_emojis.pop(emoji)
        vocab_to_index_text_emojis.update({value:emoji_index})
        emojis_list.update({emoji:value})
      except:
        continue
  all_tweets_text_emojis.append(tweet)
print(f"tweets length: for original is {len(all_tweets)} and for emojis as text is {len(all_tweets_text_emojis)}")
print(f"vocab to index size: for original is {len(vocab_to_index)} and for emojis as text is {len(vocab_to_index_text_emojis)}")

tweets length: for original is 56287 and for emojis as text is 56287
vocab to index size: for original is 9285 and for emojis as text is 9285


### vectorization

In [ ]:
vectorizer = TfidfVectorizer(
    max_df=1.0,
    ngram_range=(1,1),
    vocabulary=vocab_to_index_text_emojis
)
tfidf_list = vectorizer.fit_transform(all_tweets_text_emojis)
tfidf_list = tfidf_list.toarray()
names = vectorizer.get_feature_names_out()
print(len(names),len(vocab_to_index_text_emojis))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


9285 9285


### vectorization check

In [ ]:
# number of tweets that have '[OOV]'
oov_tweets_index = [index for index,tokenized_tweet in enumerate(tokenization_matrix) if 1 in tokenized_tweet]
print("number of tweets that have [OOV]",len(oov_tweets_index))
# subsitute tweet token with coressponded tfidf value 
index = 1
start = 0 # 0 when using train dataset and len(train_input) when using test dataset
dataset = train_input
matrix = train_data_tokenization_matrix
print('original tweet',dataset[index])
print('original tweet tokens',matrix[index])
sorted_tokenized_token = list(set(matrix[index]))
sorted_tokenized_token = [int(token) for token in sorted_tokenized_token if token != 0 and token != 1]
sorted_tokenized_token.sort(reverse = False)
print('sorted tweet tokens after removing 0,1 and duplictes',sorted_tokenized_token)
print("token corresponded tfidf value: ")
row = tfidf_list[start + index]
tfidf_dict = {}
for pos in range(len(names)):
  if names[pos] in all_tweets_text_emojis[start +index].split(" "):
    tfidf_value = row[pos]
    print(vocab_to_index_text_emojis.get(names[pos]),tfidf_value)
    tfidf_dict.update({vocab_to_index_text_emojis.get(names[pos]):tfidf_value})
print('token - tfidf dictionary',tfidf_dict)
if(len(tfidf_dict) == len(sorted_tokenized_token)):
  subsituted_tweet_tokens = [tfidf_dict.get(token) if token != 0 and token != 1 else 0 for token in matrix[index]]
  print('tweet tokens subsituted with tfidf values',subsituted_tweet_tokens)
else:
  print('token or tokens with no tfidf value')
  print('sorted tweet tokens after removing 0,1 and duplictes',sorted_tokenized_token)
  print("token corresponded tfidf value: ")

number of tweets that have [OOV] 9273
original tweet وقع اذا جات دار بشف كمل حين احس احد نقص 💔
original tweet tokens [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 141, 148, 620, 1563, 1953, 212, 186, 185, 205, 1235, 2]
sorted tweet tokens after removing 0,1 and duplictes [2, 141, 148, 185, 186, 205, 212, 620, 1235, 1563, 1953]
token corresponded tfidf value: 
2 0.15448686769909925
141 0.268936326101859
148 0.26805461209689146
185 0.27272003463292516
186 0.272198169246356
205 0.2744106313223294
212 0.2756022476933053
620 0.3234840404601987
1235 0.3626546200686223
1563 0.37680300530625127
1953 0.39253933344312447
token - tfidf dictionary {2: 0.15448686769909925, 141: 0.268936326101859, 148: 0.26805461209689146, 185: 0.27272003463292516, 186: 0.272198169246356, 205: 0.2744106313223294, 212: 0.2756022476933053, 620: 0.3234840404601987, 1235: 0.3626546200686223, 1563: 0.37680300530625127, 1953: 0.39253933344312447}
tweet tokens subsituted with tfidf values [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Encoding

### encoding function

In [ ]:
def encode(
    dataset:list, 
    matrix:list, 
    start:int, 
    index:int, 
    encoded_matrix:list, 
    error_indeces:list, 
    error_tokens:list, 
    tfidf_dict:dict
    ):
  sorted_tokenized_token = list(set(matrix[index]))
  sorted_tokenized_token = [int(token) for token in sorted_tokenized_token if token != 0 and token != 1]
  sorted_tokenized_token.sort(reverse = False)
  row = tfidf_list[start + index]
  tfidf_dict = tfidf_dict
  for pos in range(len(names)):
    if names[pos] in all_tweets_text_emojis[start +index].split(" "):
      tfidf_value = row[pos]
      tfidf_dict.update({vocab_to_index_text_emojis.get(names[pos]):tfidf_value})
  if(len(tfidf_dict) == len(sorted_tokenized_token)):
    subsituted_tweet_tokens = [tfidf_dict.get(token) if token != 0 and token != 1 else 0 for token in matrix[index]]
    encoded_matrix.append(subsituted_tweet_tokens)
  else:
    error_indeces.append(start + index)
    ini_tfidf_dict = {}
    for token in sorted_tokenized_token:
      if token not in tfidf_dict:
        error_tokens.append(token)
        ini_tfidf_dict.update({token:0.0})
    encode(
        dataset=dataset, 
        matrix=matrix, 
        start=start, 
        index=index, 
        encoded_matrix=encoded_matrix, 
        error_indeces=error_indeces, 
        error_tokens=error_tokens,
        tfidf_dict=ini_tfidf_dict
        )
    # print('error in tweet token encoding at index',index)
    # print(f'for index {start + index}')
    # print(f'for sorted tokenized token {sorted_tokenized_token}')
    # print(f'for tfidf dict {tfidf_dict}')
    # print()

def encode_token(dataset:list, matrix:list, start:int) -> list:
  encoded_matrix = []
  error_indeces = []
  error_tokens = []
  for index in range(len(matrix)):
    ini_tfidf_dict = {}
    encode(
        dataset=dataset, 
        matrix=matrix, start=start, 
        index=index, 
        encoded_matrix=encoded_matrix, 
        error_indeces=error_indeces, 
        error_tokens=error_tokens,
        tfidf_dict=ini_tfidf_dict
        )
  return encoded_matrix,error_indeces,error_tokens

In [ ]:
train_encoded_tokens_matrix,train_error_indeces,train_error_tokens = encode_token(dataset=train_input, matrix=train_data_tokenization_matrix, start=0)
test_encoded_tokens_matrix,test_error_indeces,test_error_tokens = encode_token(dataset=test_input, matrix=test_data_tokenization_matrix, start=len(train_input))


In [ ]:
error_indeces = train_error_indeces + test_error_indeces
error_tokens_index = list(set(train_error_tokens + test_error_tokens))
error_tokens_vocabs = [index_to_vocab.get(str(token)) for token in error_tokens_index]
print(f"no. of error in token-encodeing is{len(error_indeces)}")
print(f"indeces for tokens row are {error_indeces}")
print(f'no. of token with no tf-idf is {len(error_tokens_index)}')
print(f'tokens with no tf-idf are {error_tokens_vocabs,error_tokens_index}')
print(f'train tokenized tokens matrix length {len(train_data_tokenization_matrix)}')
print(f'train encoded tokens matrix length {len(train_encoded_tokens_matrix)}')
print(f'test tokenized tokens matrix length {len(test_data_tokenization_matrix)}')
print(f'test encoded tokens matrix length {len(test_encoded_tokens_matrix)}')

no. of error in token-encodeing is15
indeces for tokens row are [22584, 27795, 33554, 36443, 37475, 37675, 37809, 39766, 40411, 40888, 44491, 53163, 54961, 56114, 56173]
no. of token with no tf-idf is 2
tokens with no tf-idf are (['⏰السحب', '•⏰'], [4764, 7975])
train tokenized tokens matrix length 44861
train encoded tokens matrix length 44861
test tokenized tokens matrix length 11426
test encoded tokens matrix length 11426


In [ ]:
index = 0
print('ex. for train input dataset before and after encodeing')
print(f'encoded tokens {train_encoded_tokens_matrix[index]}')
print(f'encoded tokens length {len(train_encoded_tokens_matrix[index])}')
print(f'real tokens {train_data_tokenization_matrix[index]}')
print(f'real tokens length {len(train_data_tokenization_matrix[index])}')
print('ex. for test input dataset before and after encodeing')
print(f'encoded tokens {test_encoded_tokens_matrix[index]}')
print(f'encoded tokens length {len(test_encoded_tokens_matrix[index])}')
print(f'real tokens {test_data_tokenization_matrix[index]}')
print(f'real tokens length{len(test_data_tokenization_matrix[index])}')

ex. for train input dataset before and after encodeing
encoded tokens [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2008758878190134, 0.18872435011640437, 0.35472829282851, 0.2868684683572954, 0.5469554531880279, 0.5469554531880279, 0.38915912594795826, 0.32135897603536173, 0.2138846529422816, 0.29329436788329316, 0.17395804649362281]
encoded tokens length 26
real tokens [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 38, 26, 2325, 670, 476, 476, 3906, 1357, 68, 778, 8]
real tokens length 26
ex. for test input dataset before and after encodeing
encoded tokens [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.580742894652676, 0.580742894652676, 0.2869169032177081, 0.4385284873291479, 0, 0.22549243239991712]
encoded tokens length 26
real tokens [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9156, 8340, 98, 1833, 1, 8]
real tokens length26


### save encoding matrix

In [ ]:
encoding_list = dict({
    'test_data_encoding_matrix':test_encoded_tokens_matrix,
    'train_data_encoding_matrix':train_encoded_tokens_matrix
})
encoding_matrix_path = os.path.join(data_folder,'encoding_matrix.json')

with open(encoding_matrix_path, "wt+") as f:
    json.dump(obj=encoding_list, fp=f)